In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [4]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [5]:
import os
from langchain_ollama import ChatOllama
from langchain.agents import create_agent

model_name="granite4:1b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url
)

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt
)

In [6]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [12]:
from pprint import pprint

pprint(response['messages'][-1].content)

('The **LangChain MCP Adapters** is a package that makes it easy to use '
 '**Anthropic Model Context Protocol (MCP) tools** with LangChain & '
 'LangGraph.<|fim_suffix|>\n'
 '  - Supports multiple tool servers across different MCP servers.\n'
 '  - Allows agents to pull from multiple MCP servers at once for more '
 'powerful applications.\n'
 '\n'
 '### Why Use MCP Adapters:\n'
 '- Makes it simple to connect LangChain and LangGraph with the growing '
 'ecosystem of MCP tool servers.\n'
 '- Instead of manually adapting each tool, you can integrate them '
 'seamlessly.\n'
 '- Enables agents to pull from multiple MCP servers at once, making it easier '
 'to combine different tools for more powerful applications.\n'
 '\n'
 '### Why Use MCP Adapters:\n'
 '- **Ease of Integration:** No need to write adapter code or manage '
 'individual MCP server instances. Just specify which tool servers to connect '
 'to and how they should be invoked.\n'
 '- **Centralized Tool Management:** Easily manag

## Online MCP

In [16]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=Europe/Madrid"
            ]
        }
    }
)

tools = await client.get_tools()

In [17]:
agent = create_agent(
    model=model,
    tools=tools,
)

In [19]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response['messages'][-1].content)

'It is currently Thursday, December 25, 2025 at **03:09** in Madrid (UTC+1).'
